In [ ]:
"""
Created on Mon Jul 10 18:47 2023

Use cdo to regrid the variables to 4km stereographic grid

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Transformer
import pandas as pd
import sys,os
from cdo import Cdo
import time
from tqdm.notebook import tqdm

In [ ]:
%matplotlib qt5

In [ ]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

PREPARE PATH AND CONTEXT

In [ ]:
homepath='/bettik/burgardc'

mod='CNRM-CM6-1'
scenario='historical'
ens_run='r1i1p1f2'

In [ ]:
path1='/bettik/jourdai1/OCEAN_DATA_CMIP6_STEREO/'+mod+'/'
path2=homepath+'/DATA/SUMMER_PAPER/raw/CMIP_DATA/'+mod+'/'
path3=homepath+'/DATA/SUMMER_PAPER/interim/'

In [ ]:
ISMIP_4km = xr.open_dataset(path3+'ISMIP6_AIS_4000m_grid.nc')

In [ ]:
ISMIP_8km = ISMIP_4km.sel(x=ISMIP_4km.x[::2],y=ISMIP_4km.y[::2])

In [ ]:
ISMIP_8km['cell_area'] = ISMIP_8km['cell_area']*0 + (8000. * 8000.)

In [ ]:
ISMIP_8km.to_netcdf(path3+'ISMIP6_AIS_8000m_grid_Clara.nc')

In [ ]:
Nico_CMIP = xr.open_dataset(path1+'thetao_Oyr_'+mod+'_'+scenario+'_'+ens_run+'_185001_194912.nc')

In [ ]:
Nico_CMIP

REGRIDDING THETA

In [ ]:
time_start = time.time()
cdo.remapbil(path3+'ISMIP6_AIS_4000m_grid.nc', 
             input = path1+'thetao_Oyr_'+mod+'_'+scenario+'_'+ens_run+'_185001_194912.nc', 
             output = path2+'thetao_Oyr_'+mod+'_'+scenario+'_'+ens_run+'_185001_194912_4km.nc')
timelength = time.time() - time_start

REGRIDDING THE MELT AND MAKING LAND NAN FOR OTHER VARIABLES

In [ ]:
file_tmask_3D = xr.open_dataset(outputpath + 'mask_variables_of_interest_Ant.nc')
for yy in tqdm(range(yy_start,yy_end+1)):
    var_of_int = xr.open_dataset(outputpath + 'variables_of_interest_'+str(yy)+'_Ant_wolandmask.nc')
    
    var_of_int = var_of_int.rename({'y_grid_T': 'y','x_grid_T': 'x','deptht': 'nav_lev' })
    var_of_int[['thetao','so']].where(file_tmask_3D['tmask_2']).rename({
    'nav_lev': 'deptht'}).to_netcdf(outputpath + 'variables_of_interest_'+str(yy)+'_Ant.nc')

    melt_check = var_of_int['fwfisf'].where(file_tmask_3D['tmask'])
    var_of_int['melt_cavity'] = melt_check.where(np.isfinite(melt_check),0)
    check2 = -1*var_of_int[['melt_cavity']]
    check2.to_netcdf(outputpath+'cavity_melt_'+str(yy)+'_Ant.nc')
    
    

In [ ]:
for yy in tqdm(range(yy_start,yy_end+1)):
    #print('computing ',yy)
    time_start = time.time()
    cdo.remapbil(outputpath+'stereo_grid.nc', input = outputpath+'cavity_melt_'+str(yy)+'_Ant.nc', output = outputpath+'cavity_melt_'+str(yy)+'_Ant_stereo.nc')
    timelength = time.time() - time_start
    #print(timelength)

REGRIDDING ALL OTHER VARIABLES OF INTEREST

In [ ]:
#for yy in tqdm(range(yy_start,yy_end+1)): #restart at 1998
for yy in tqdm(range(2093,yy_end+1)): #
    print('computing ',yy)
    time_start = time.time()
    cdo.remapbil(outputpath+'stereo_grid.nc', input = outputpath+'variables_of_interest_'+str(yy)+'_Ant.nc', output = outputpath+'variables_of_interest_'+str(yy)+'_Ant_stereo.nc')
    timelength = time.time() - time_start
    print(timelength)

FINISHED? GOOD! THEN YOU ARE READY TO GO TO THE PRE-PROCESSING OF THE MASKS AND TEMPERATURE & SALINITY PROFILES =)